In [1]:
import pandas as pd
import numpy as np

In [ ]:
sleep_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Sleep.csv"
)
sleep_df.head()

,user_id,resp_time,datetime_utc,sleep_hours,sleep_quality,sleepiness,location
0,u00,1364114365,2013-03-24 08:39:25+00:00,NaN,NaN,NaN,NaN
1,u00,1364114453,2013-03-24 08:40:53+00:00,NaN,NaN,NaN,NaN
2,u00,1364114760,2013-03-24 08:46:00+00:00,NaN,NaN,NaN,NaN
3,u00,1364114761,2013-03-24 08:46:01+00:00,NaN,NaN,NaN,NaN
4,u00,1364114765,2013-03-24 08:46:05+00:00,NaN,NaN,NaN,NaN


In [ ]:
sleep_df["datetime_utc"] = pd.to_datetime(sleep_df["datetime_utc"])
sleep_df["date_only"] = sleep_df["datetime_utc"].dt.date
sleep_df = sleep_df.drop(columns=["location", "resp_time"])
print(sleep_df.head(20))

   user_id              datetime_utc  sleep_hours  sleep_quality  sleepiness  \
0      u00 2013-03-24 08:39:25+00:00          NaN            NaN         NaN   
1      u00 2013-03-24 08:40:53+00:00          NaN            NaN         NaN   
2      u00 2013-03-24 08:46:00+00:00          NaN            NaN         NaN   
3      u00 2013-03-24 08:46:01+00:00          NaN            NaN         NaN   
4      u00 2013-03-24 08:46:05+00:00          NaN            NaN         NaN   
5      u00 2013-03-24 08:46:11+00:00          NaN            NaN         NaN   
6      u00 2013-03-24 08:46:15+00:00          NaN            NaN         NaN   
7      u00 2013-03-24 18:07:40+00:00          NaN            NaN         NaN   
8      u00 2013-03-25 18:54:36+00:00          8.0            1.0         1.0   
9      u00 2013-03-26 18:07:14+00:00          7.0            2.0         1.0   
10     u00 2013-03-26 21:46:45+00:00          8.0            1.0         1.0   
11     u00 2013-03-27 04:09:49+00:00    

In [4]:
agg_sleep_df = sleep_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_sleep_df.head()

,user_id,date_only,datetime_utc,sleep_hours,sleep_quality,sleepiness
0,u00,2013-03-24,2013-03-24 09:54:48.750000128+00:00,NaN,NaN,NaN
1,u00,2013-03-25,2013-03-25 18:54:36+00:00,8.0,1.0,1.0
2,u00,2013-03-26,2013-03-26 19:56:59.500000+00:00,7.5,1.5,1.0
3,u00,2013-03-27,2013-03-27 11:14:08+00:00,8.0,2.0,1.0
4,u00,2013-03-28,2013-03-28 09:42:53.500000+00:00,6.5,2.0,1.0


In [ ]:
# create a new table
users = agg_sleep_df["user_id"].unique()
min_date = agg_sleep_df["date_only"].min()
max_date = agg_sleep_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_sleep_df["date_only"] = pd.to_datetime(agg_sleep_df["date_only"])
merged_sleep_df = pd.merge(
    full_df, agg_sleep_df, on=["user_id", "date_only"], how="left"
)
merged_sleep_df = merged_sleep_df.drop(columns=["datetime_utc"])
merged_sleep_df.head(20)

,user_id,date_only,sleep_hours,sleep_quality,sleepiness
0,u00,2013-03-24,NaN,NaN,NaN
1,u00,2013-03-25,8.0,1.0,1.0
2,u00,2013-03-26,7.5,1.5,1.0
3,u00,2013-03-27,8.0,2.0,1.0
4,u00,2013-03-28,6.5,2.0,1.0
5,u00,2013-03-29,6.0,2.0,1.0
6,u00,2013-03-30,NaN,NaN,NaN
7,u00,2013-03-31,9.0,2.0,1.0
8,u00,2013-04-01,3.0,4.0,1.0
9,u00,2013-04-02,NaN,NaN,NaN


In [ ]:
def recursive_fill_missing(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ["sleep_hours", "sleep_quality", "sleepiness"]:
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()

            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break
    return group


filled_sleep_df = (
    merged_sleep_df.groupby("user_id")
    .apply(recursive_fill_missing)
    .reset_index(drop=True)
)
filled_sleep_df.head(20)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/3132625382.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_sleep_df = merged_sleep_df.groupby("user_id").apply(recursive_fill_missing).reset_index(drop=True)


,user_id,date_only,sleep_hours,sleep_quality,sleepiness
0,u00,2013-03-24,NaN,NaN,NaN
1,u00,2013-03-25,8.00,1.0,1.0
2,u00,2013-03-26,7.50,1.5,1.0
3,u00,2013-03-27,8.00,2.0,1.0
4,u00,2013-03-28,6.50,2.0,1.0
5,u00,2013-03-29,6.00,2.0,1.0
6,u00,2013-03-30,6.25,2.0,1.0
7,u00,2013-03-31,9.00,2.0,1.0
8,u00,2013-04-01,3.00,4.0,1.0
9,u00,2013-04-02,6.00,3.0,1.0


In [ ]:
# import matplotlib.pyplot as plt

# filled_sleep_df = filled_sleep_df.sort_values(by=["user_id", "date_only"])

# user_list = filled_sleep_df["user_id"].unique()

# for user in user_list:
#     user_df = filled_sleep_df[filled_sleep_df["user_id"] == user]

#     plt.figure(figsize=(12, 6))
#     plt.plot(user_df["date_only"], user_df["sleep_hours"], label="Sleep Hours", marker='o')
#     plt.plot(user_df["date_only"], user_df["sleep_quality"], label="Sleep Quality", marker='s')
#     plt.plot(user_df["date_only"], user_df["sleepiness"], label="Sleepiness", marker='^')

#     plt.title(f"Sleep Trends for {user}")
#     plt.xlabel("Date")
#     plt.ylabel("Score / Hours")
#     plt.xticks(rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.grid(True)
#     plt.show()

In [ ]:
social_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Social.csv"
)
social_df.head(20)

,user_id,resp_time,datetime_utc,number,location
0,u00,1364114390,2013-03-24 08:39:50+00:00,NaN,NaN
1,u00,1364114393,2013-03-24 08:39:53+00:00,NaN,NaN
2,u00,1364114455,2013-03-24 08:40:55+00:00,NaN,NaN
3,u00,1364163095,2013-03-24 22:11:35+00:00,NaN,NaN
4,u00,1364177775,2013-03-25 02:16:15+00:00,NaN,NaN
5,u00,1364264483,2013-03-26 02:21:23+00:00,5.0,Unknown
6,u00,1364328798,2013-03-26 20:13:18+00:00,1.0,"43.70697406,-72.28758978"
7,u00,1364338301,2013-03-26 22:51:41+00:00,1.0,"43.70759062,-72.28510298"
8,u00,1364425743,2013-03-27 23:09:03+00:00,4.0,"43.70863158,-72.2842741"
9,u00,1364437520,2013-03-28 02:25:20+00:00,3.0,"43.70517792,-72.28685304"


In [ ]:
social_df["datetime_utc"] = pd.to_datetime(social_df["datetime_utc"])
social_df["date_only"] = social_df["datetime_utc"].dt.date
social_df = social_df.drop(columns=["location", "resp_time"])
print(social_df.head(20))

   user_id              datetime_utc  number   date_only
0      u00 2013-03-24 08:39:50+00:00     NaN  2013-03-24
1      u00 2013-03-24 08:39:53+00:00     NaN  2013-03-24
2      u00 2013-03-24 08:40:55+00:00     NaN  2013-03-24
3      u00 2013-03-24 22:11:35+00:00     NaN  2013-03-24
4      u00 2013-03-25 02:16:15+00:00     NaN  2013-03-25
5      u00 2013-03-26 02:21:23+00:00     5.0  2013-03-26
6      u00 2013-03-26 20:13:18+00:00     1.0  2013-03-26
7      u00 2013-03-26 22:51:41+00:00     1.0  2013-03-26
8      u00 2013-03-27 23:09:03+00:00     4.0  2013-03-27
9      u00 2013-03-28 02:25:20+00:00     3.0  2013-03-28
10     u00 2013-03-29 05:14:08+00:00     4.0  2013-03-29
11     u00 2013-03-29 06:00:01+00:00     4.0  2013-03-29
12     u00 2013-03-29 16:03:43+00:00     4.0  2013-03-29
13     u00 2013-03-30 06:07:28+00:00     4.0  2013-03-30
14     u00 2013-03-31 06:11:24+00:00     2.0  2013-03-31
15     u00 2013-04-01 07:41:16+00:00     2.0  2013-04-01
16     u00 2013-04-03 04:57:43+

In [10]:
agg_social_df = social_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_social_df.head()

,user_id,date_only,datetime_utc,number
0,u00,2013-03-24,2013-03-24 12:03:03.249999872+00:00,NaN
1,u00,2013-03-25,2013-03-25 02:16:15+00:00,NaN
2,u00,2013-03-26,2013-03-26 15:08:47.333333248+00:00,2.333333
3,u00,2013-03-27,2013-03-27 23:09:03+00:00,4.000000
4,u00,2013-03-28,2013-03-28 02:25:20+00:00,3.000000


In [ ]:
# create a new table
users = agg_social_df["user_id"].unique()
min_date = agg_social_df["date_only"].min()
max_date = agg_social_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_social_df["date_only"] = pd.to_datetime(agg_social_df["date_only"])
merged_social_df = pd.merge(
    full_df, agg_social_df, on=["user_id", "date_only"], how="left"
)
merged_social_df = merged_social_df.drop(columns=["datetime_utc"])
merged_social_df.head(20)

,user_id,date_only,number
0,u00,2013-03-24,NaN
1,u00,2013-03-25,NaN
2,u00,2013-03-26,2.333333
3,u00,2013-03-27,4.000000
4,u00,2013-03-28,3.000000
5,u00,2013-03-29,4.000000
6,u00,2013-03-30,4.000000
7,u00,2013-03-31,2.000000
8,u00,2013-04-01,2.000000
9,u00,2013-04-02,NaN


In [ ]:
def recursive_fill_missing(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ["number"]:
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()

            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break
    return group


filled_social_df = (
    merged_social_df.groupby("user_id")
    .apply(recursive_fill_missing)
    .reset_index(drop=True)
)
filled_social_df.head(20)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/2718863653.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_social_df = merged_social_df.groupby("user_id").apply(recursive_fill_missing).reset_index(drop=True)


,user_id,date_only,number
0,u00,2013-03-24,NaN
1,u00,2013-03-25,NaN
2,u00,2013-03-26,2.333333
3,u00,2013-03-27,4.000000
4,u00,2013-03-28,3.000000
5,u00,2013-03-29,4.000000
6,u00,2013-03-30,4.000000
7,u00,2013-03-31,2.000000
8,u00,2013-04-01,2.000000
9,u00,2013-04-02,2.000000


In [ ]:
# import matplotlib.pyplot as plt

# filled_social_df = filled_social_df.sort_values(by=["user_id", "date_only"])

# user_list = filled_social_df["user_id"].unique()

# for user in user_list:
#     user_df = filled_social_df[filled_social_df["user_id"] == user]

#     plt.figure(figsize=(12, 6))
#     plt.plot(user_df["date_only"], user_df["number"], label="number", marker='o')

#     plt.title(f"Social Trends for {user}")
#     plt.xlabel("Date")
#     plt.ylabel("Score / Hours")
#     plt.xticks(rotation=45)
#     plt.legend()
#     plt.tight_layout()
#     plt.grid(True)
#     plt.show()

In [ ]:
stress_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Stress_new.csv"
)
stress_df.head(20)

,user_id,level,location,resp_time,datetime_utc
0,u00,2,"43.70692415,-72.2873929",1364237696,2013-03-25 18:54:56+00:00
1,u00,2,"43.70555193,-72.28704778",1364268806,2013-03-26 03:33:26+00:00
2,u00,2,"43.70555193,-72.28704778",1364268814,2013-03-26 03:33:34+00:00
3,u00,1,"43.70678675,-72.28732051",1364346740,2013-03-27 01:12:20+00:00
4,u00,1,"43.70508322,-72.28677496",1364437527,2013-03-28 02:25:27+00:00
5,u00,2,"43.70514505,-72.28786512",1364496192,2013-03-28 18:43:12+00:00
6,u00,3,"43.70492199,-72.28765189",1364498245,2013-03-28 19:17:25+00:00
7,u00,2,"43.70541477,-72.28837166",1364498754,2013-03-28 19:25:54+00:00
8,u00,4,"43.70540958,-72.28769817",1364504894,2013-03-28 21:08:14+00:00
9,u00,4,"43.70032217,-72.28941584",1364509152,2013-03-28 22:19:12+00:00


In [ ]:
stress_df["datetime_utc"] = pd.to_datetime(stress_df["datetime_utc"])
stress_df["date_only"] = stress_df["datetime_utc"].dt.date
stress_df = stress_df.drop(columns=["location", "resp_time"])
print(stress_df.head(20))

   user_id  level              datetime_utc   date_only
0      u00      2 2013-03-25 18:54:56+00:00  2013-03-25
1      u00      2 2013-03-26 03:33:26+00:00  2013-03-26
2      u00      2 2013-03-26 03:33:34+00:00  2013-03-26
3      u00      1 2013-03-27 01:12:20+00:00  2013-03-27
4      u00      1 2013-03-28 02:25:27+00:00  2013-03-28
5      u00      2 2013-03-28 18:43:12+00:00  2013-03-28
6      u00      3 2013-03-28 19:17:25+00:00  2013-03-28
7      u00      2 2013-03-28 19:25:54+00:00  2013-03-28
8      u00      4 2013-03-28 21:08:14+00:00  2013-03-28
9      u00      4 2013-03-28 22:19:12+00:00  2013-03-28
10     u00      4 2013-03-29 02:05:54+00:00  2013-03-29
11     u00      1 2013-03-29 15:57:22+00:00  2013-03-29
12     u00      1 2013-03-29 20:54:17+00:00  2013-03-29
13     u00      4 2013-03-29 23:08:06+00:00  2013-03-29
14     u00      4 2013-03-30 04:19:24+00:00  2013-03-30
15     u00      4 2013-03-31 17:54:45+00:00  2013-03-31
16     u00      5 2013-03-31 22:55:27+00:00  201

In [16]:
agg_stress_df = stress_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_stress_df.head()

,user_id,date_only,level,datetime_utc
0,u00,2013-03-25,2.000000,2013-03-25 18:54:56+00:00
1,u00,2013-03-26,2.000000,2013-03-26 03:33:30+00:00
2,u00,2013-03-27,1.000000,2013-03-27 01:12:20+00:00
3,u00,2013-03-28,2.666667,2013-03-28 17:13:14+00:00
4,u00,2013-03-29,2.500000,2013-03-29 15:31:24.750000128+00:00


In [ ]:
# create a new table
users = agg_stress_df["user_id"].unique()
min_date = agg_stress_df["date_only"].min()
max_date = agg_stress_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_stress_df["date_only"] = pd.to_datetime(agg_stress_df["date_only"])
merged_stress_df = pd.merge(
    full_df, agg_stress_df, on=["user_id", "date_only"], how="left"
)
merged_stress_df = merged_stress_df.drop(columns=["datetime_utc"])
merged_stress_df.head(20)

,user_id,date_only,level
0,u00,2013-03-25,2.000000
1,u00,2013-03-26,2.000000
2,u00,2013-03-27,1.000000
3,u00,2013-03-28,2.666667
4,u00,2013-03-29,2.500000
5,u00,2013-03-30,4.000000
6,u00,2013-03-31,4.500000
7,u00,2013-04-01,NaN
8,u00,2013-04-02,1.000000
9,u00,2013-04-03,1.500000


In [ ]:
def recursive_fill_missing(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ["level"]:
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()

            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break
    return group


filled_stress_df = (
    merged_stress_df.groupby("user_id")
    .apply(recursive_fill_missing)
    .reset_index(drop=True)
)
filled_stress_df.head(20)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/1250882267.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_stress_df = merged_stress_df.groupby("user_id").apply(recursive_fill_missing).reset_index(drop=True)


,user_id,date_only,level
0,u00,2013-03-25,2.000000
1,u00,2013-03-26,2.000000
2,u00,2013-03-27,1.000000
3,u00,2013-03-28,2.666667
4,u00,2013-03-29,2.500000
5,u00,2013-03-30,4.000000
6,u00,2013-03-31,4.500000
7,u00,2013-04-01,4.250000
8,u00,2013-04-02,1.000000
9,u00,2013-04-03,1.500000


In [ ]:
mood_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Mood_new.csv"
)
mood_df.head(20)

,user_id,happy,happyornot,location,resp_time,sad,sadornot,datetime_utc
0,u00,2,1.0,Unknown,1366857633,4,1.0,2013-04-25 02:40:33+00:00
1,u00,1,2.0,"43.75921839,-72.32919741",1366870195,3,1.0,2013-04-25 06:09:55+00:00
2,u00,3,1.0,"43.75880824,-72.32925862",1368631462,1,2.0,2013-05-15 15:24:22+00:00
3,u00,2,1.0,"43.71332016,-72.30876457",1368718987,1,1.0,2013-05-16 15:43:07+00:00
4,u00,2,1.0,"43.75929012,-72.32904032",1368802750,1,NaN,2013-05-17 14:59:10+00:00
5,u00,1,NaN,"43.75906255,-72.32936718",1369104096,1,NaN,2013-05-21 02:41:36+00:00
6,u00,1,2.0,"43.70685166,-72.28720734",1369195078,1,2.0,2013-05-22 03:57:58+00:00
7,u00,1,1.0,"43.7065519,-72.28701015",1369276811,1,1.0,2013-05-23 02:40:11+00:00
8,u00,1,NaN,"43.70206745,-72.2892032",1375840039,1,NaN,2013-08-07 01:47:19+00:00
9,u00,1,NaN,"43.70579481,-72.28577693",1376106249,1,NaN,2013-08-10 03:44:09+00:00


In [ ]:
mood_df["datetime_utc"] = pd.to_datetime(stress_df["datetime_utc"])
mood_df["date_only"] = mood_df["datetime_utc"].dt.date
mood_df = mood_df.drop(columns=["location", "resp_time"])
print(mood_df.head(20))

   user_id  happy  happyornot  sad  sadornot              datetime_utc  \
0      u00      2         1.0    4       1.0 2013-03-25 18:54:56+00:00   
1      u00      1         2.0    3       1.0 2013-03-26 03:33:26+00:00   
2      u00      3         1.0    1       2.0 2013-03-26 03:33:34+00:00   
3      u00      2         1.0    1       1.0 2013-03-27 01:12:20+00:00   
4      u00      2         1.0    1       NaN 2013-03-28 02:25:27+00:00   
5      u00      1         NaN    1       NaN 2013-03-28 18:43:12+00:00   
6      u00      1         2.0    1       2.0 2013-03-28 19:17:25+00:00   
7      u00      1         1.0    1       1.0 2013-03-28 19:25:54+00:00   
8      u00      1         NaN    1       NaN 2013-03-28 21:08:14+00:00   
9      u00      1         NaN    1       NaN 2013-03-28 22:19:12+00:00   
10     u01      2         1.0    1       1.0 2013-03-29 02:05:54+00:00   
11     u01      2         1.0    1       1.0 2013-03-29 15:57:22+00:00   
12     u02      1         2.0    1    

In [21]:
agg_mood_df = mood_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_mood_df.head()

,user_id,date_only,happy,happyornot,sad,sadornot,datetime_utc
0,u00,2013-03-25,2.000000,1.000000,4.0,1.0,2013-03-25 18:54:56+00:00
1,u00,2013-03-26,2.000000,1.500000,2.0,1.5,2013-03-26 03:33:30+00:00
2,u00,2013-03-27,2.000000,1.000000,1.0,1.0,2013-03-27 01:12:20+00:00
3,u00,2013-03-28,1.166667,1.333333,1.0,1.5,2013-03-28 17:13:14+00:00
4,u01,2013-03-29,2.000000,1.000000,1.0,1.0,2013-03-29 09:01:38+00:00


In [ ]:
# create a new table
users = mood_df["user_id"].unique()
min_date = mood_df["date_only"].min()
max_date = mood_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_mood_df["date_only"] = pd.to_datetime(agg_social_df["date_only"])
merged_mood_df = pd.merge(full_df, agg_mood_df, on=["user_id", "date_only"], how="left")
merged_mood_df = merged_mood_df.drop(columns=["datetime_utc"])
merged_mood_df.head(20)

,user_id,date_only,happy,happyornot,sad,sadornot
0,u00,2013-03-25,2.000000,1.500000,2.0,1.5
1,u00,2013-03-26,2.000000,1.000000,1.0,1.0
2,u00,2013-03-27,1.166667,1.333333,1.0,1.5
3,u00,2013-03-28,NaN,NaN,NaN,NaN
4,u00,2013-03-29,NaN,NaN,NaN,NaN
5,u00,2013-03-30,NaN,NaN,NaN,NaN
6,u00,2013-03-31,NaN,NaN,NaN,NaN
7,u00,2013-04-01,NaN,NaN,NaN,NaN
8,u00,2013-04-02,NaN,NaN,NaN,NaN
9,u00,2013-04-03,NaN,NaN,NaN,NaN


In [ ]:
def recursive_fill_mood(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ["happyornot", "happy", "sadornot", "sad"]:
            group[col] = pd.to_numeric(group[col], errors="coerce")

            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()

            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break
    return group


filled_mood_df = (
    mood_df.groupby("user_id").apply(recursive_fill_mood).reset_index(drop=True)
)

filled_mood_df["happyornot"] = (
    filled_mood_df["happyornot"].round().fillna(0).astype(int)
)
filled_mood_df["sadornot"] = filled_mood_df["sadornot"].round().fillna(0).astype(int)

filled_mood_df.head(20)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/1883587671.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_mood_df = mood_df.groupby("user_id").apply(recursive_fill_mood).reset_index(drop=True)


,user_id,happy,happyornot,sad,sadornot,datetime_utc,date_only
0,u00,2,1,4,1,2013-03-25 18:54:56+00:00,2013-03-25
1,u00,1,2,3,1,2013-03-26 03:33:26+00:00,2013-03-26
2,u00,3,1,1,2,2013-03-26 03:33:34+00:00,2013-03-26
3,u00,2,1,1,1,2013-03-27 01:12:20+00:00,2013-03-27
4,u00,2,1,1,2,2013-03-28 02:25:27+00:00,2013-03-28
5,u00,1,1,1,1,2013-03-28 18:43:12+00:00,2013-03-28
6,u00,1,2,1,2,2013-03-28 19:17:25+00:00,2013-03-28
7,u00,1,1,1,1,2013-03-28 19:25:54+00:00,2013-03-28
8,u00,1,2,1,2,2013-03-28 21:08:14+00:00,2013-03-28
9,u00,1,1,1,1,2013-03-28 22:19:12+00:00,2013-03-28


In [ ]:
behavior_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Behaviour_new.csv"
)
behavior_df.head(20)

,user_id,anxious,calm,conventional,critical,dependable,disorganized,enthusiastic,experiences,reserved,resp_time,sympathetic,datetime_utc
0,u00,4,4.0,5.0,2,3.0,3.0,4.0,4.0,3.0,1365455068,4.0,2013-04-08 21:04:28+00:00
1,u00,5,3.0,3.0,1,4.0,4.0,4.0,3.0,1.0,1365621587,4.0,2013-04-10 19:19:47+00:00
2,u00,4,3.0,3.0,3,4.0,2.0,4.0,4.0,2.0,1366145919,4.0,2013-04-16 20:58:39+00:00
3,u00,4,5.0,2.0,1,4.0,3.0,5.0,3.0,1.0,1366229877,4.0,2013-04-17 20:17:57+00:00
4,u00,3,3.0,3.0,3,3.0,3.0,3.0,4.0,3.0,1366663342,4.0,2013-04-22 20:42:22+00:00
5,u00,5,3.0,4.0,5,3.0,4.0,2.0,1.0,3.0,1366867587,4.0,2013-04-25 05:26:27+00:00
6,u00,5,3.0,4.0,2,5.0,1.0,4.0,3.0,1.0,1367008911,3.0,2013-04-26 20:41:51+00:00
7,u00,3,NaN,NaN,3,3.0,3.0,3.0,3.0,3.0,1368576414,3.0,2013-05-15 00:06:54+00:00
8,u00,3,2.0,2.0,3,3.0,2.0,3.0,2.0,2.0,1368662695,2.0,2013-05-16 00:04:55+00:00
9,u00,4,4.0,3.0,2,3.0,3.0,2.0,3.0,5.0,1368837399,4.0,2013-05-18 00:36:39+00:00


In [25]:
behavior_df["datetime_utc"] = pd.to_datetime(stress_df["datetime_utc"])
behavior_df["date_only"] = behavior_df["datetime_utc"].dt.date

behavior_df = behavior_df.drop(columns=["resp_time"])
print(behavior_df.head(20))

   user_id  anxious  calm  conventional  critical  dependable  disorganized  \
0      u00        4   4.0           5.0         2         3.0           3.0   
1      u00        5   3.0           3.0         1         4.0           4.0   
2      u00        4   3.0           3.0         3         4.0           2.0   
3      u00        4   5.0           2.0         1         4.0           3.0   
4      u00        3   3.0           3.0         3         3.0           3.0   
5      u00        5   3.0           4.0         5         3.0           4.0   
6      u00        5   3.0           4.0         2         5.0           1.0   
7      u00        3   NaN           NaN         3         3.0           3.0   
8      u00        3   2.0           2.0         3         3.0           2.0   
9      u00        4   4.0           3.0         2         3.0           3.0   
10     u00        3   3.0           2.0         3         3.0           3.0   
11     u00        4   2.0           3.0         5   

In [ ]:
agg_behavior_df = behavior_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_behavior_df[
    [
        "anxious",
        "calm",
        "conventional",
        "enthusiastic",
        "reserved",
        "critical",
        "sympathetic",
        "dependable",
        "disorganized",
        "experiences",
    ]
] = agg_behavior_df[
    [
        "anxious",
        "calm",
        "conventional",
        "enthusiastic",
        "reserved",
        "critical",
        "sympathetic",
        "dependable",
        "disorganized",
        "experiences",
    ]
].round()
agg_behavior_df.head(10)

,user_id,date_only,anxious,calm,conventional,critical,dependable,disorganized,enthusiastic,experiences,reserved,sympathetic,datetime_utc
0,u00,2013-03-25,4.0,4.0,5.0,2.0,3.0,3.0,4.0,4.0,3.0,4.0,2013-03-25 18:54:56+00:00
1,u00,2013-03-26,4.0,3.0,3.0,2.0,4.0,3.0,4.0,4.0,2.0,4.0,2013-03-26 03:33:30+00:00
2,u00,2013-03-27,4.0,5.0,2.0,1.0,4.0,3.0,5.0,3.0,1.0,4.0,2013-03-27 01:12:20+00:00
3,u00,2013-03-28,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2013-03-28 17:13:14+00:00
4,u00,2013-03-29,3.0,2.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0,2013-03-29 12:59:11+00:00
5,u01,2013-03-29,1.0,5.0,5.0,1.0,3.0,2.0,3.0,3.0,5.0,3.0,2013-03-29 23:08:06+00:00
6,u02,2013-03-30,1.0,5.0,2.0,2.0,2.0,3.0,5.0,2.0,2.0,4.0,2013-03-30 04:19:24+00:00
7,u02,2013-03-31,2.0,4.0,3.0,3.0,3.0,1.0,5.0,4.0,2.0,5.0,2013-03-31 17:54:45+00:00
8,u03,2013-03-31,1.0,5.0,3.0,4.0,4.0,1.0,3.0,5.0,4.0,3.0,2013-03-31 22:55:27+00:00
9,u03,2013-04-02,3.0,3.0,3.0,3.0,5.0,1.0,3.0,4.0,4.0,3.0,2013-04-02 22:17:41+00:00


In [ ]:
# create a new table
users = behavior_df["user_id"].unique()
min_date = behavior_df["date_only"].min()
max_date = behavior_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_behavior_df["date_only"] = pd.to_datetime(agg_behavior_df["date_only"])
merged_behavior_df = pd.merge(
    full_df, agg_behavior_df, on=["user_id", "date_only"], how="left"
)
merged_behavior_df = merged_behavior_df.drop(columns=["datetime_utc"])
merged_behavior_df.head(20)

,user_id,date_only,anxious,calm,conventional,critical,dependable,disorganized,enthusiastic,experiences,reserved,sympathetic
0,u00,2013-03-25,4.0,4.0,5.0,2.0,3.0,3.0,4.0,4.0,3.0,4.0
1,u00,2013-03-26,4.0,3.0,3.0,2.0,4.0,3.0,4.0,4.0,2.0,4.0
2,u00,2013-03-27,4.0,5.0,2.0,1.0,4.0,3.0,5.0,3.0,1.0,4.0
3,u00,2013-03-28,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,u00,2013-03-29,3.0,2.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,2.0
5,u00,2013-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,u00,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,u00,2013-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,u00,2013-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,u00,2013-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ordinal_cols = [
    "anxious",
    "calm",
    "conventional",
    "enthusiastic",
    "reserved",
    "critical",
    "sympathetic",
    "dependable",
    "disorganized",
    "experiences",
]


def recursive_fill_behavior(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ordinal_cols:
            group[col] = pd.to_numeric(group[col], errors="coerce")
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()
            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break

    group[ordinal_cols] = group[ordinal_cols].round().astype("Int64")

    return group


filled_behavior_df = (
    behavior_df.groupby("user_id").apply(recursive_fill_behavior).reset_index(drop=True)
)
filled_behavior_df.head(20)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/512093721.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_behavior_df = behavior_df.groupby("user_id").apply(


,user_id,anxious,calm,conventional,critical,dependable,disorganized,enthusiastic,experiences,reserved,sympathetic,datetime_utc,date_only
0,u00,4,4,5,2,3,3,4,4,3,4,2013-03-25 18:54:56+00:00,2013-03-25
1,u00,5,3,3,1,4,4,4,3,1,4,2013-03-26 03:33:26+00:00,2013-03-26
2,u00,4,3,3,3,4,2,4,4,2,4,2013-03-26 03:33:34+00:00,2013-03-26
3,u00,4,5,2,1,4,3,5,3,1,4,2013-03-27 01:12:20+00:00,2013-03-27
4,u00,3,3,3,3,3,3,3,4,3,4,2013-03-28 02:25:27+00:00,2013-03-28
5,u00,5,3,4,5,3,4,2,1,3,4,2013-03-28 18:43:12+00:00,2013-03-28
6,u00,5,3,4,2,5,1,4,3,1,3,2013-03-28 19:17:25+00:00,2013-03-28
7,u00,3,3,4,3,3,3,3,3,3,3,2013-03-28 19:25:54+00:00,2013-03-28
8,u00,3,2,2,3,3,2,3,2,2,2,2013-03-28 21:08:14+00:00,2013-03-28
9,u00,4,4,3,2,3,3,2,3,5,4,2013-03-28 22:19:12+00:00,2013-03-28


In [ ]:
class_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Class_new.csv"
)
class_df.head(20)

,user_id,course_id,due,experience,hours,location,resp_time,datetime_utc
0,u00,cs65\n,2,3,1,"43.70544533,-72.28679012",1364271993,2013-03-26 04:26:33+00:00
1,u00,NaN,1,1,1,"43.75932116,-72.32890406",1364357271,2013-03-27 04:07:51+00:00
2,u00,c65,2,3,2,"43.70465788,-72.2887224",1364429426,2013-03-28 00:10:26+00:00
3,u00,cs65,2,2,1,"43.70033175,-72.28949659",1364509180,2013-03-28 22:19:40+00:00
4,u00,cs65,2,3,2,"43.70688502,-72.28755706",1364516771,2013-03-29 00:26:11+00:00
5,u00,cs65,2,3,1,"43.70777831,-72.28510534",1364583727,2013-03-29 19:02:07+00:00
6,u00,cs66,2,3,3,"43.70587194,-72.28717655",1364601962,2013-03-30 00:06:02+00:00
7,u00,NaN,2,1,1,"43.76243366,-72.32615434",1364779548,2013-04-01 01:25:48+00:00
8,u00,cs65,1,2,4,"43.70534371,-72.28812527",1364857571,2013-04-01 23:06:11+00:00
9,u00,cs65\n,2,3,3,Unknown,1364940107,2013-04-02 22:01:47+00:00


In [ ]:
class_df["datetime_utc"] = pd.to_datetime(stress_df["datetime_utc"])
class_df["date_only"] = class_df["datetime_utc"].dt.date
class_df = class_df.drop(columns=["location", "resp_time", "course_id"])
print(class_df.head(20))

   user_id  due  experience  hours              datetime_utc   date_only
0      u00    2           3      1 2013-03-25 18:54:56+00:00  2013-03-25
1      u00    1           1      1 2013-03-26 03:33:26+00:00  2013-03-26
2      u00    2           3      2 2013-03-26 03:33:34+00:00  2013-03-26
3      u00    2           2      1 2013-03-27 01:12:20+00:00  2013-03-27
4      u00    2           3      2 2013-03-28 02:25:27+00:00  2013-03-28
5      u00    2           3      1 2013-03-28 18:43:12+00:00  2013-03-28
6      u00    2           3      3 2013-03-28 19:17:25+00:00  2013-03-28
7      u00    2           1      1 2013-03-28 19:25:54+00:00  2013-03-28
8      u00    1           2      4 2013-03-28 21:08:14+00:00  2013-03-28
9      u00    2           3      3 2013-03-28 22:19:12+00:00  2013-03-28
10     u00    2           3      7 2013-03-29 02:05:54+00:00  2013-03-29
11     u00    1           2      6 2013-03-29 15:57:22+00:00  2013-03-29
12     u00    2           3      6 2013-03-29 20:54

In [31]:
agg_class_df = class_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_class_df.head()

,user_id,date_only,due,experience,hours,datetime_utc
0,u00,2013-03-25,2.000000,3.00,1.000000,2013-03-25 18:54:56+00:00
1,u00,2013-03-26,1.500000,2.00,1.500000,2013-03-26 03:33:30+00:00
2,u00,2013-03-27,2.000000,2.00,1.000000,2013-03-27 01:12:20+00:00
3,u00,2013-03-28,1.833333,2.50,2.333333,2013-03-28 17:13:14+00:00
4,u00,2013-03-29,1.750000,2.75,6.500000,2013-03-29 15:31:24.750000128+00:00


In [ ]:
def recursive_fill_class(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ["due", "experience", "hours"]:
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()
            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break
    group["due"] = group["due"].round(0).astype("Int64")
    group["experience"] = group["experience"].round(0).astype("Int64")
    group["hours"] = group["hours"].round(0).astype("Int64")

    return group


filled_class_df = (
    agg_class_df.groupby("user_id").apply(recursive_fill_class).reset_index(drop=True)
)
filled_class_df.head()

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/1036472677.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_class_df = agg_class_df.groupby("user_id").apply(recursive_fill_class).reset_index(drop=True)


,user_id,date_only,due,experience,hours,datetime_utc
0,u00,2013-03-25,2,3,1,2013-03-25 18:54:56+00:00
1,u00,2013-03-26,2,2,2,2013-03-26 03:33:30+00:00
2,u00,2013-03-27,2,2,1,2013-03-27 01:12:20+00:00
3,u00,2013-03-28,2,2,2,2013-03-28 17:13:14+00:00
4,u00,2013-03-29,2,3,6,2013-03-29 15:31:24.750000128+00:00


In [ ]:
events_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Events.csv"
)
events_df.head(20)

,user_id,resp_time,datetime_utc,positive_event_score,negative_event_score,emotion_range,has_positive_text,has_negative_text,positive_text,negative_text,location
0,u46,1366313506,2013-04-18 19:31:46+00:00,4,2,2,0,0,NaN,NaN,"43.70418499,-72.29773208"
1,u46,1367035341,2013-04-27 04:02:21+00:00,4,6,-2,0,0,NaN,NaN,"44.45507574,-73.18376895"
2,u50,1370291657,2013-06-03 20:34:17+00:00,4,1,3,0,0,NaN,NaN,Unknown
3,u07,1366392235,2013-04-19 17:23:55+00:00,5,5,0,1,1,Networks lab,Negative: mit shooting,"43.70375498,-72.28370087"
4,u07,1366752684,2013-04-23 21:31:24+00:00,5,6,-1,1,1,Finishing lab3,Controversy,"43.70425769,-72.28391582"
5,u27,1366316880,2013-04-18 20:28:00+00:00,3,3,0,1,1,djs meeting,?,"43.70424431,-72.28559775"
6,u27,1366390948,2013-04-19 17:02:28+00:00,3,4,-1,0,0,NaN,NaN,"43.70375531,-72.28604079"
7,u27,1366770604,2013-04-24 02:30:04+00:00,1,6,-5,0,0,NaN,NaN,"43.69949922,-72.28974367"
8,u27,1367122704,2013-04-28 04:18:24+00:00,1,1,0,0,0,NaN,NaN,"43.70552478,-72.28974476"
9,u27,1366956080,2013-04-26 06:01:20+00:00,1,1,0,0,0,NaN,NaN,"43.70396926,-72.28577776"


In [ ]:
events_df["datetime_utc"] = pd.to_datetime(stress_df["datetime_utc"])
events_df["date_only"] = events_df["datetime_utc"].dt.date

events_df = events_df.drop(
    columns=[
        "resp_time",
        "positive_text",
        "negative_text",
        "has_positive_text",
        "has_negative_text",
        "location",
    ]
)
print(events_df.head(20))

   user_id              datetime_utc  positive_event_score  \
0      u46 2013-03-25 18:54:56+00:00                     4   
1      u46 2013-03-26 03:33:26+00:00                     4   
2      u50 2013-03-26 03:33:34+00:00                     4   
3      u07 2013-03-27 01:12:20+00:00                     5   
4      u07 2013-03-28 02:25:27+00:00                     5   
5      u27 2013-03-28 18:43:12+00:00                     3   
6      u27 2013-03-28 19:17:25+00:00                     3   
7      u27 2013-03-28 19:25:54+00:00                     1   
8      u27 2013-03-28 21:08:14+00:00                     1   
9      u27 2013-03-28 22:19:12+00:00                     1   
10     u27 2013-03-29 02:05:54+00:00                     1   
11     u27 2013-03-29 15:57:22+00:00                     1   
12     u30 2013-03-29 20:54:17+00:00                     5   
13     u51 2013-03-29 23:08:06+00:00                     4   
14     u51 2013-03-30 04:19:24+00:00                     1   
15     u

In [ ]:
agg_events_df = events_df.groupby(["user_id", "date_only"], as_index=False).mean()
agg_events_df[["positive_event_score", "negative_event_score", "emotion_range"]] = (
    agg_events_df[
        ["positive_event_score", "negative_event_score", "emotion_range"]
    ].round()
)
agg_events_df.head(10)

,user_id,date_only,datetime_utc,positive_event_score,negative_event_score,emotion_range
0,u00,2013-05-20,2013-05-20 04:03:56+00:00,5.0,5.0,0.0
1,u00,2013-05-21,2013-05-21 15:57:18+00:00,6.0,3.0,3.0
2,u00,2013-05-22,2013-05-22 14:38:41.666666752+00:00,2.0,7.0,-5.0
3,u00,2013-05-23,2013-05-23 18:12:52.500000+00:00,6.0,4.0,2.0
4,u00,2013-05-24,2013-05-24 14:17:32+00:00,4.0,6.0,-2.0
5,u00,2013-05-25,2013-05-25 10:41:54+00:00,3.0,4.0,-1.0
6,u00,2013-06-03,2013-06-03 06:00:04+00:00,3.0,4.0,-1.0
7,u00,2013-08-07,2013-08-07 01:48:04+00:00,4.0,7.0,-3.0
8,u00,2013-08-08,2013-08-08 00:47:33+00:00,1.0,1.0,0.0
9,u00,2013-08-10,2013-08-10 03:44:40+00:00,1.0,1.0,0.0


In [ ]:
# create a new table
users = events_df["user_id"].unique()
min_date = events_df["date_only"].min()
max_date = events_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_events_df["date_only"] = pd.to_datetime(agg_events_df["date_only"])
merged_events_df = pd.merge(
    full_df, agg_events_df, on=["user_id", "date_only"], how="left"
)
merged_events_df = merged_events_df.drop(columns=["datetime_utc"])
merged_events_df.head(20)

,user_id,date_only,positive_event_score,negative_event_score,emotion_range
0,u46,2013-03-25,4.0,2.0,2.0
1,u46,2013-03-26,4.0,6.0,-2.0
2,u46,2013-03-27,NaN,NaN,NaN
3,u46,2013-03-28,NaN,NaN,NaN
4,u46,2013-03-29,NaN,NaN,NaN
5,u46,2013-03-30,NaN,NaN,NaN
6,u46,2013-03-31,NaN,NaN,NaN
7,u46,2013-04-01,NaN,NaN,NaN
8,u46,2013-04-02,NaN,NaN,NaN
9,u46,2013-04-03,NaN,NaN,NaN


In [ ]:
def recursive_fill_events(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    for _ in range(max_iter):
        all_filled = True
        for col in ["positive_event_score", "negative_event_score", "emotion_range"]:
            group[col] = pd.to_numeric(group[col], errors="coerce")
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()
            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False
        if all_filled:
            break
    for col in ["positive_event_score", "negative_event_score", "emotion_range"]:
        group[col] = group[col].round().astype("Int64")

    return group


filled_events_df = (
    events_df.groupby("user_id").apply(recursive_fill_events).reset_index(drop=True)
)

filled_events_df.head(20)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/2001948664.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_events_df = events_df.groupby("user_id").apply(


,user_id,datetime_utc,positive_event_score,negative_event_score,emotion_range,date_only
0,u00,2013-05-20 04:03:56+00:00,5,5,0,2013-05-20
1,u00,2013-05-21 15:57:18+00:00,6,3,3,2013-05-21
2,u00,2013-05-22 04:11:19+00:00,2,7,-5,2013-05-22
3,u00,2013-05-22 19:41:14+00:00,2,7,-5,2013-05-22
4,u00,2013-05-22 20:03:32+00:00,2,6,-4,2013-05-22
5,u00,2013-05-23 17:06:35+00:00,4,4,0,2013-05-23
6,u00,2013-05-23 19:19:10+00:00,7,3,4,2013-05-23
7,u00,2013-05-24 21:26:10+00:00,3,7,-4,2013-05-24
8,u00,2013-05-24 07:08:54+00:00,4,4,0,2013-05-24
9,u00,2013-05-25 04:00:14+00:00,3,6,-3,2013-05-25


In [ ]:
exercise_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/EMA/Exercise.csv"
)
exercise_df.head(20)

,user_id,resp_time,weekday,hour,exercise,walk,have,schedule,latitude,longitude,has_location
0,u00,2013-04-01T14:55:38+00:00,Monday,14,3,2.0,2.0,1.0,NaN,NaN,0
1,u00,2013-04-01T17:12:56+00:00,Monday,17,1,1.0,2.0,1.0,NaN,NaN,0
2,u00,2013-04-02T04:03:53+00:00,Tuesday,4,1,1.0,1.0,NaN,NaN,NaN,0
3,u00,2013-04-03T03:43:26+00:00,Wednesday,3,4,2.0,1.0,NaN,NaN,NaN,0
4,u00,2013-04-04T00:28:03+00:00,Thursday,0,1,2.0,2.0,1.0,NaN,NaN,0
5,u00,2013-04-04T02:03:44+00:00,Thursday,2,1,3.0,2.0,1.0,NaN,NaN,0
6,u00,2013-04-05T02:39:51+00:00,Friday,2,4,3.0,1.0,2.0,NaN,NaN,0
7,u00,2013-04-05T20:37:51+00:00,Friday,20,4,3.0,1.0,NaN,NaN,NaN,0
8,u00,2013-04-06T02:19:21+00:00,Saturday,2,1,1.0,2.0,1.0,NaN,NaN,0
9,u00,2013-04-07T01:56:12+00:00,Sunday,1,3,3.0,1.0,NaN,NaN,NaN,0


In [ ]:
exercise_df["datetime_utc"] = pd.to_datetime(exercise_df["resp_time"])
exercise_df["date_only"] = exercise_df["datetime_utc"].dt.date

exercise_df = exercise_df.drop(
    columns=["resp_time", "weekday", "latitude", "longitude", "has_location"]
)
print(exercise_df.head(20))

   user_id  hour  exercise  walk  have  schedule              datetime_utc  \
0      u00    14         3   2.0   2.0       1.0 2013-04-01 14:55:38+00:00   
1      u00    17         1   1.0   2.0       1.0 2013-04-01 17:12:56+00:00   
2      u00     4         1   1.0   1.0       NaN 2013-04-02 04:03:53+00:00   
3      u00     3         4   2.0   1.0       NaN 2013-04-03 03:43:26+00:00   
4      u00     0         1   2.0   2.0       1.0 2013-04-04 00:28:03+00:00   
5      u00     2         1   3.0   2.0       1.0 2013-04-04 02:03:44+00:00   
6      u00     2         4   3.0   1.0       2.0 2013-04-05 02:39:51+00:00   
7      u00    20         4   3.0   1.0       NaN 2013-04-05 20:37:51+00:00   
8      u00     2         1   1.0   2.0       1.0 2013-04-06 02:19:21+00:00   
9      u00     1         3   3.0   1.0       NaN 2013-04-07 01:56:12+00:00   
10     u00    18         3   3.0   2.0       2.0 2013-04-07 18:35:08+00:00   
11     u00     5         1   3.0   2.0       1.0 2013-04-10 05:2

In [40]:
agg_exercise_df = exercise_df.groupby(["user_id", "date_only"], as_index=False).mean()

agg_exercise_df.head(10)

,user_id,date_only,hour,exercise,walk,have,schedule,datetime_utc
0,u00,2013-04-01,15.5,2.0,1.5,2.0,1.0,2013-04-01 16:04:17+00:00
1,u00,2013-04-02,4.0,1.0,1.0,1.0,NaN,2013-04-02 04:03:53+00:00
2,u00,2013-04-03,3.0,4.0,2.0,1.0,NaN,2013-04-03 03:43:26+00:00
3,u00,2013-04-04,1.0,1.0,2.5,2.0,1.0,2013-04-04 01:15:53.500000+00:00
4,u00,2013-04-05,11.0,4.0,3.0,1.0,2.0,2013-04-05 11:38:51+00:00
5,u00,2013-04-06,2.0,1.0,1.0,2.0,1.0,2013-04-06 02:19:21+00:00
6,u00,2013-04-07,9.5,3.0,3.0,1.5,2.0,2013-04-07 10:15:40+00:00
7,u00,2013-04-10,5.0,1.0,3.0,2.0,1.0,2013-04-10 05:24:58+00:00
8,u00,2013-04-12,22.0,1.0,3.0,2.0,1.0,2013-04-12 22:54:04+00:00
9,u00,2013-04-17,11.0,1.0,1.5,2.0,1.0,2013-04-17 11:36:13+00:00


In [ ]:
# create a new table
users = exercise_df["user_id"].unique()
min_date = exercise_df["date_only"].min()
max_date = exercise_df["date_only"].max()
full_dates = pd.date_range(start=min_date, end=max_date)
full_index = pd.MultiIndex.from_product(
    [users, full_dates], names=["user_id", "date_only"]
)
full_df = pd.DataFrame(index=full_index).reset_index()
agg_exercise_df["date_only"] = pd.to_datetime(agg_exercise_df["date_only"])
merged_exercise_df = pd.merge(
    full_df, agg_exercise_df, on=["user_id", "date_only"], how="left"
)
merged_exercise_df = merged_exercise_df.drop(columns=["datetime_utc"])
merged_exercise_df.head(20)

,user_id,date_only,hour,exercise,walk,have,schedule
0,u00,2013-04-01,15.5,2.0,1.5,2.0,1.0
1,u00,2013-04-02,4.0,1.0,1.0,1.0,NaN
2,u00,2013-04-03,3.0,4.0,2.0,1.0,NaN
3,u00,2013-04-04,1.0,1.0,2.5,2.0,1.0
4,u00,2013-04-05,11.0,4.0,3.0,1.0,2.0
5,u00,2013-04-06,2.0,1.0,1.0,2.0,1.0
6,u00,2013-04-07,9.5,3.0,3.0,1.5,2.0
7,u00,2013-04-08,NaN,NaN,NaN,NaN,NaN
8,u00,2013-04-09,NaN,NaN,NaN,NaN,NaN
9,u00,2013-04-10,5.0,1.0,3.0,2.0,1.0


In [ ]:
def recursive_fill_exercise(group, max_iter=100):
    group = group.sort_values("date_only").copy()

    ordinal_cols = ["exercise", "walk", "have", "schedule"]
    numeric_cols = ["hour"]

    for _ in range(max_iter):
        all_filled = True

        for col in ordinal_cols:
            group[col] = pd.to_numeric(group[col], errors="coerce")
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()
            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False

        for col in numeric_cols:
            group[col] = pd.to_numeric(group[col], errors="coerce")
            rolling_avg = group[col].rolling(window=3, min_periods=1).mean()
            mask = group[col].isna()
            if mask.any():
                group.loc[mask, col] = rolling_avg[mask]
                all_filled = False

        if all_filled:
            break

    for col in ordinal_cols:
        group[col] = group[col].round().astype("Int64")

    return group


filled_exercise_df = (
    merged_exercise_df.groupby("user_id")
    .apply(recursive_fill_exercise)
    .reset_index(drop=True)
)

filled_exercise_df.head(10)

/var/folders/wv/mc7gb7ls1r7_nb4hd3kbdmg80000gn/T/ipykernel_76986/273583958.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_exercise_df = merged_exercise_df.groupby("user_id").apply(


,user_id,date_only,hour,exercise,walk,have,schedule
0,u00,2013-04-01,15.50,2,2,2,1
1,u00,2013-04-02,4.00,1,1,1,1
2,u00,2013-04-03,3.00,4,2,1,1
3,u00,2013-04-04,1.00,1,2,2,1
4,u00,2013-04-05,11.00,4,3,1,2
5,u00,2013-04-06,2.00,1,1,2,1
6,u00,2013-04-07,9.50,3,3,2,2
7,u00,2013-04-08,5.75,2,2,2,2
8,u00,2013-04-09,9.50,3,3,2,2
9,u00,2013-04-10,5.00,1,3,2,1


In [ ]:
def standardize_date_format(df):
    df["date_only"] = pd.to_datetime(df["date_only"])
    return df

In [ ]:
filled_sleep_df = standardize_date_format(filled_sleep_df)
filled_class_df = standardize_date_format(filled_class_df)
filled_social_df = standardize_date_format(filled_social_df)
filled_behavior_df = standardize_date_format(filled_behavior_df)
filled_events_df = standardize_date_format(filled_events_df)
filled_exercise_df = standardize_date_format(filled_exercise_df)
filled_mood_df = standardize_date_format(filled_mood_df)
filled_stress_df = standardize_date_format(filled_stress_df)

In [ ]:
from functools import reduce
import pandas as pd


def combine_dataframes(dfs, merge_keys=["user_id", "date_only"]):
    def drop_conflicting_columns(df):
        return df.drop(
            columns=[col for col in df.columns if "datetime_utc" in col],
            errors="ignore",
        )

    def preprocess_df(df):

        df = drop_conflicting_columns(df)

        df = df.groupby(merge_keys, as_index=False).mean(numeric_only=True)
        return df

    dfs_cleaned = [preprocess_df(df) for df in dfs]

    merged_df = reduce(
        lambda left, right: pd.merge(left, right, on=merge_keys, how="outer"),
        dfs_cleaned,
    )

    merged_df = merged_df.sort_values(by=merge_keys).reset_index(drop=True)

    return merged_df


dfs = [
    filled_sleep_df,
    filled_class_df,
    filled_social_df,
    filled_behavior_df,
    filled_events_df,
    filled_exercise_df,
    filled_mood_df,
    filled_stress_df,
]

final_merged_df = combine_dataframes(dfs)
final_merged_df.head(20)

,user_id,date_only,sleep_hours,sleep_quality,sleepiness,due,experience,hours,number,anxious,...,hour,exercise,walk,have,schedule,happy,happyornot,sad,sadornot,level
0,u00,2013-03-24,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN
1,u00,2013-03-25,8.00,1.0,1.0,2.0,3.0,1.0,NaN,4.0,...,NaN,<NA>,<NA>,<NA>,<NA>,2.000000,1.000000,4.0,1.0,2.000000
2,u00,2013-03-26,7.50,1.5,1.0,2.0,2.0,2.0,2.333333,4.5,...,NaN,<NA>,<NA>,<NA>,<NA>,2.000000,1.500000,2.0,1.5,2.000000
3,u00,2013-03-27,8.00,2.0,1.0,2.0,2.0,1.0,4.000000,4.0,...,NaN,<NA>,<NA>,<NA>,<NA>,2.000000,1.000000,1.0,1.0,1.000000
4,u00,2013-03-28,6.50,2.0,1.0,2.0,2.0,2.0,3.000000,3.833333,...,NaN,<NA>,<NA>,<NA>,<NA>,1.166667,1.333333,1.0,1.5,2.666667
5,u00,2013-03-29,6.00,2.0,1.0,2.0,3.0,6.0,4.000000,3.0,...,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,2.500000
6,u00,2013-03-30,6.25,2.0,1.0,1.0,3.0,7.0,4.000000,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,4.000000
7,u00,2013-03-31,9.00,2.0,1.0,2.0,2.0,6.0,2.000000,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,4.500000
8,u00,2013-04-01,3.00,4.0,1.0,<NA>,<NA>,<NA>,2.000000,<NA>,...,15.50,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,4.250000
9,u00,2013-04-02,6.00,3.0,1.0,1.0,1.0,9.0,2.000000,<NA>,...,4.00,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.000000


In [ ]:
# combine the education variables
grades_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/education/grades.csv"
)
grades_df = grades_df[["uid", "gpa 13s"]].rename(
    columns={"uid": "user_id", "gpa 13s": "gpa_13s"}
)
grades_df.head(10)

,user_id,gpa_13s
0,u01,1.777778
1,u02,4.000000
2,u04,3.500000
3,u05,3.777778
4,u07,4.000000
5,u08,3.333333
6,u09,3.777778
7,u10,3.777778
8,u12,2.583333
9,u14,3.888889


In [137]:
final_merged_df = final_merged_df.merge(grades_df, on="user_id", how="left")
final_merged_df.head(10)

,user_id,date_only,sleep_hours,sleep_quality,sleepiness,due,experience,hours,number,anxious,...,exercise,walk,have,schedule,happy,happyornot,sad,sadornot,level,gpa_13s
0,u00,2013-03-24,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
1,u00,2013-03-25,8.00,1.0,1.0,2.0,3.0,1.0,NaN,4.0,...,<NA>,<NA>,<NA>,<NA>,2.000000,1.000000,4.0,1.0,2.000000,NaN
2,u00,2013-03-26,7.50,1.5,1.0,2.0,2.0,2.0,2.333333,4.5,...,<NA>,<NA>,<NA>,<NA>,2.000000,1.500000,2.0,1.5,2.000000,NaN
3,u00,2013-03-27,8.00,2.0,1.0,2.0,2.0,1.0,4.000000,4.0,...,<NA>,<NA>,<NA>,<NA>,2.000000,1.000000,1.0,1.0,1.000000,NaN
4,u00,2013-03-28,6.50,2.0,1.0,2.0,2.0,2.0,3.000000,3.833333,...,<NA>,<NA>,<NA>,<NA>,1.166667,1.333333,1.0,1.5,2.666667,NaN
5,u00,2013-03-29,6.00,2.0,1.0,2.0,3.0,6.0,4.000000,3.0,...,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,2.500000,NaN
6,u00,2013-03-30,6.25,2.0,1.0,1.0,3.0,7.0,4.000000,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,4.000000,NaN
7,u00,2013-03-31,9.00,2.0,1.0,2.0,2.0,6.0,2.000000,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,4.500000,NaN
8,u00,2013-04-01,3.00,4.0,1.0,<NA>,<NA>,<NA>,2.000000,<NA>,...,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,4.250000,NaN
9,u00,2013-04-02,6.00,3.0,1.0,1.0,1.0,9.0,2.000000,<NA>,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.000000,NaN


In [ ]:
# add deadline variable
deadline_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/raw_data/education/deadlines.csv"
)
deadline_df.head(20)

,uid,2013-03-27,2013-03-28,2013-03-29,2013-03-30,2013-03-31,2013-04-01,2013-04-02,2013-04-03,2013-04-04,...,2013-08-10,2013-08-11,2013-08-12,2013-08-13,2013-08-14,2013-08-15,2013-08-16,2013-08-17,2013-08-18,2013-08-19
0,u01,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,u02,0,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,u03,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,u04,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,u05,0,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,u07,0,0,0,0,0,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,u08,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,u09,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,u10,0,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,u12,1,0,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
deadline_long_df = deadline_df.melt(
    id_vars="uid", var_name="date_only", value_name="has_deadline"
)


deadline_long_df = deadline_long_df.rename(columns={"uid": "user_id"})


deadline_long_df["date_only"] = pd.to_datetime(deadline_long_df["date_only"])


final_merged_df = final_merged_df.merge(
    deadline_long_df, on=["user_id", "date_only"], how="left"
)

final_merged_df["has_deadline"] = final_merged_df["has_deadline"].fillna(0)
final_merged_df.head(10)

,user_id,date_only,sleep_hours,sleep_quality,sleepiness,due,experience,hours,number,anxious,...,walk,have,schedule,happy,happyornot,sad,sadornot,level,gpa_13s,has_deadline
0,u00,2013-03-24,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,u00,2013-03-25,8.00,1.0,1.0,2.0,3.0,1.0,NaN,4.0,...,<NA>,<NA>,<NA>,2.000000,1.000000,4.0,1.0,2.000000,NaN,0.0
2,u00,2013-03-26,7.50,1.5,1.0,2.0,2.0,2.0,2.333333,4.5,...,<NA>,<NA>,<NA>,2.000000,1.500000,2.0,1.5,2.000000,NaN,0.0
3,u00,2013-03-27,8.00,2.0,1.0,2.0,2.0,1.0,4.000000,4.0,...,<NA>,<NA>,<NA>,2.000000,1.000000,1.0,1.0,1.000000,NaN,0.0
4,u00,2013-03-28,6.50,2.0,1.0,2.0,2.0,2.0,3.000000,3.833333,...,<NA>,<NA>,<NA>,1.166667,1.333333,1.0,1.5,2.666667,NaN,0.0
5,u00,2013-03-29,6.00,2.0,1.0,2.0,3.0,6.0,4.000000,3.0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,2.500000,NaN,0.0
6,u00,2013-03-30,6.25,2.0,1.0,1.0,3.0,7.0,4.000000,<NA>,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,4.000000,NaN,0.0
7,u00,2013-03-31,9.00,2.0,1.0,2.0,2.0,6.0,2.000000,<NA>,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,4.500000,NaN,0.0
8,u00,2013-04-01,3.00,4.0,1.0,<NA>,<NA>,<NA>,2.000000,<NA>,...,2.0,2.0,1.0,NaN,NaN,NaN,NaN,4.250000,NaN,0.0
9,u00,2013-04-02,6.00,3.0,1.0,1.0,1.0,9.0,2.000000,<NA>,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.000000,NaN,0.0


In [ ]:
# combine the final_merged_df with phq total score
phq_df = pd.read_csv(
    "https://raw.githubusercontent.com/fanxu30/student_mental_health_ml/refs/heads/main/cleaned_data/surveys/clean_PHQ-9.csv"
)
phq_df_post = phq_df[phq_df["type"] == "post"][["uid", "total_score"]]
phq_df_post = phq_df_post[["uid", "total_score"]].rename(
    columns={"uid": "user_id", "total_score": "total_score"}
)
phq_df_post.head(20)

,user_id,total_score
46,u00,3
47,u01,4
48,u02,5
49,u03,4
50,u04,8
51,u05,0
52,u07,8
53,u09,2
54,u10,4
55,u14,3


In [141]:
final_merged_df = final_merged_df.merge(phq_df_post, on="user_id", how="left")
final_merged_df.head(10)

,user_id,date_only,sleep_hours,sleep_quality,sleepiness,due,experience,hours,number,anxious,...,have,schedule,happy,happyornot,sad,sadornot,level,gpa_13s,has_deadline,total_score
0,u00,2013-03-24,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0
1,u00,2013-03-25,8.00,1.0,1.0,2.0,3.0,1.0,NaN,4.0,...,<NA>,<NA>,2.000000,1.000000,4.0,1.0,2.000000,NaN,0.0,3.0
2,u00,2013-03-26,7.50,1.5,1.0,2.0,2.0,2.0,2.333333,4.5,...,<NA>,<NA>,2.000000,1.500000,2.0,1.5,2.000000,NaN,0.0,3.0
3,u00,2013-03-27,8.00,2.0,1.0,2.0,2.0,1.0,4.000000,4.0,...,<NA>,<NA>,2.000000,1.000000,1.0,1.0,1.000000,NaN,0.0,3.0
4,u00,2013-03-28,6.50,2.0,1.0,2.0,2.0,2.0,3.000000,3.833333,...,<NA>,<NA>,1.166667,1.333333,1.0,1.5,2.666667,NaN,0.0,3.0
5,u00,2013-03-29,6.00,2.0,1.0,2.0,3.0,6.0,4.000000,3.0,...,<NA>,<NA>,NaN,NaN,NaN,NaN,2.500000,NaN,0.0,3.0
6,u00,2013-03-30,6.25,2.0,1.0,1.0,3.0,7.0,4.000000,<NA>,...,<NA>,<NA>,NaN,NaN,NaN,NaN,4.000000,NaN,0.0,3.0
7,u00,2013-03-31,9.00,2.0,1.0,2.0,2.0,6.0,2.000000,<NA>,...,<NA>,<NA>,NaN,NaN,NaN,NaN,4.500000,NaN,0.0,3.0
8,u00,2013-04-01,3.00,4.0,1.0,<NA>,<NA>,<NA>,2.000000,<NA>,...,2.0,1.0,NaN,NaN,NaN,NaN,4.250000,NaN,0.0,3.0
9,u00,2013-04-02,6.00,3.0,1.0,1.0,1.0,9.0,2.000000,<NA>,...,1.0,1.0,NaN,NaN,NaN,NaN,1.000000,NaN,0.0,3.0


In [142]:
final_merged_df.columns

Index(['user_id', 'date_only', 'sleep_hours', 'sleep_quality', 'sleepiness',
       'due', 'experience', 'hours', 'number', 'anxious', 'calm',
       'conventional', 'critical', 'dependable', 'disorganized',
       'enthusiastic', 'experiences', 'reserved', 'sympathetic',
       'positive_event_score', 'negative_event_score', 'emotion_range', 'hour',
       'exercise', 'walk', 'have', 'schedule', 'happy', 'happyornot', 'sad',
       'sadornot', 'level', 'gpa_13s', 'has_deadline', 'total_score'],
      dtype='object')

In [ ]:
# LSTM
input_features = [
    "sleep_hours",
    "sleep_quality",
    "sleepiness",
    "due",
    "experience",
    "hours",
    "number",
    "anxious",
    "calm",
    "conventional",
    "critical",
    "dependable",
    "disorganized",
    "enthusiastic",
    "experiences",
    "reserved",
    "sympathetic",
    "positive_event_score",
    "negative_event_score",
    "emotion_range",
    "hour",
    "exercise",
    "walk",
    "have",
    "schedule",
    "happy",
    "happyornot",
    "sad",
    "sadornot",
    "level",
    "gpa_13s",
    "has_deadline",
]

In [ ]:
print(final_merged_df["total_score"].isna().sum())

1536


In [ ]:
final_merged_df[input_features + ["total_score"]].isna().sum()

sleep_hours             3607
sleep_quality           3607
sleepiness              3607
due                     6460
experience              6460
hours                   6460
number                  3673
anxious                 6843
calm                    6845
conventional            6845
critical                6843
dependable              6843
disorganized            6845
enthusiastic            6843
experiences             6844
reserved                6844
sympathetic             6844
positive_event_score    6913
negative_event_score    6913
emotion_range           6913
hour                    1231
exercise                1231
walk                    1231
have                    1231
schedule                1271
happy                   6909
happyornot              6909
sad                     6909
sadornot                6909
level                    309
gpa_13s                 2704
has_deadline               0
total_score             1536
dtype: int64

In [ ]:
clean_df = final_merged_df.dropna(subset=input_features + ["total_score"])
print(f"Remaining rows: {len(clean_df)}")

Remaining rows: 0


In [ ]:
import numpy as np


def create_lstm_sequences(df, input_cols, target_col="total_score", seq_len=3):
    sequences = []
    targets = []

    df = df.sort_values(by=["user_id", "date_only"])

    for user_id, group in df.groupby("user_id"):
        group = group.dropna(subset=input_cols + [target_col])

        values = group[input_cols].values
        labels = group[target_col].values

        for i in range(len(group) - seq_len):
            sequences.append(values[i : i + seq_len])
            targets.append(labels[i + seq_len])

    return np.array(sequences), np.array(targets)

In [ ]:
X, y = create_lstm_sequences(
    final_merged_df, input_cols=input_features, target_col="total_score", seq_len=3
)

In [ ]:
print(f"Total samples: {len(X)}")

Total samples: 0


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


set_seed(42)


class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.125, random_state=42
)

train_loader = DataLoader(
    SequenceDataset(X_train, y_train), batch_size=32, shuffle=True
)
val_loader = DataLoader(SequenceDataset(X_val, y_val), batch_size=32)
test_loader = DataLoader(SequenceDataset(X_test, y_test), batch_size=32)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.